# Sleep Data Analysis

In [1]:
import pandas as pd

## The dataset is semicolon separated

In [2]:
sleep_data_until_2019 = pd.read_csv('sleepdata.csv',sep=";")

In [3]:
sleep_data_until_2019

,Start,End,Sleep quality,Time in bed,Wake up,Sleep Notes,Heart rate,Activity (steps)
0,2014-12-29 22:57:49,2014-12-30 07:30:13,100%,8:32,:),NaN,59.0,0
1,2014-12-30 21:17:50,2014-12-30 21:33:54,3%,0:16,:|,Stressful day,72.0,0
2,2014-12-30 22:42:49,2014-12-31 07:13:31,98%,8:30,:|,NaN,57.0,0
3,2014-12-31 22:31:01,2015-01-01 06:03:01,65%,7:32,NaN,NaN,NaN,0
4,2015-01-01 22:12:10,2015-01-02 04:56:35,72%,6:44,:),Drank coffee:Drank tea,68.0,0
...,...,...,...,...,...,...,...,...
882,2018-02-12 21:54:14,2018-02-13 07:02:15,91%,9:08,NaN,NaN,NaN,56
883,2018-02-13 23:49:19,2018-02-14 07:00:53,81%,7:11,NaN,NaN,NaN,64
884,2018-02-14 21:24:05,2018-02-15 06:20:52,71%,8:56,NaN,NaN,NaN,3316
885,2018-02-15 21:36:32,2018-02-16 06:50:31,80%,9:13,NaN,NaN,NaN,6555


## Check data types

In [4]:
sleep_data_until_2019.dtypes

Start                object
End                  object
Sleep quality        object
Time in bed          object
Wake up              object
Sleep Notes          object
Heart rate          float64
Activity (steps)      int64
dtype: object

Start and end columns are supposed to be DateTime objects. Sleep quality is supposed to be a number not a percent. Wake up is supposed to be imputed.

## Converting the datatypes

Change the datatypes to dates

In [5]:
def change_dtypes(df, columns, result_types):
    for i in range(len(columns)):
        if result_types[i] == 'date':
            df[columns[i]] = pd.to_datetime(df[columns[i]])
        else:
            df[columns[i]] = df[columns[i]].astype(result_types[i])
change_dtypes(sleep_data_until_2019,['Start','End'],['date','date'])

Change the sleep quality to a float.

In [6]:
sleep_data_until_2019['Sleep quality'] = sleep_data_until_2019['Sleep quality'].str.rstrip('%').astype('float')

Change the time in bed to number of minutes in bed.

In [7]:
sleep_data_until_2019['Hours in bed'] = sleep_data_until_2019['Time in bed'].str.split(':').str[0]
sleep_data_until_2019['Minutes in bed'] = sleep_data_until_2019['Time in bed'].str.split(':').str[1]

In [8]:
sleep_data_until_2019['Time in bed'] = sleep_data_until_2019['Hours in bed'].astype(int) * 60 + sleep_data_until_2019['Minutes in bed'].astype(int)
sleep_data_until_2019 = sleep_data_until_2019.drop(['Hours in bed','Minutes in bed'],axis = 1)

Change the sleep notes into variables.

In [9]:
sleep_data_until_2019['Sleep Notes'] = sleep_data_until_2019['Sleep Notes'].fillna("None")

In [10]:
sleep_data_until_2019['Stressful day'] = sleep_data_until_2019['Sleep Notes'].str.contains('Stressful day').astype(int)
sleep_data_until_2019['Worked out'] = sleep_data_until_2019['Sleep Notes'].str.contains('Worked out').astype(int)
sleep_data_until_2019['Drank tea'] = sleep_data_until_2019['Sleep Notes'].str.contains('Drank tea').astype(int)
sleep_data_until_2019['Drank coffee'] = sleep_data_until_2019['Sleep Notes'].str.contains('Drank coffee').astype(int)
sleep_data_until_2019['Ate late'] = sleep_data_until_2019['Sleep Notes'].str.contains('Ate late').astype(int)
sleep_data_until_2019 = sleep_data_until_2019.drop('Sleep Notes',axis = 1)

Detect null values in the dataset

In [11]:
sleep_data_until_2019.isnull().sum()

Start                 0
End                   0
Sleep quality         0
Time in bed           0
Wake up             641
Heart rate          725
Activity (steps)      0
Stressful day         0
Worked out            0
Drank tea             0
Drank coffee          0
Ate late              0
dtype: int64

Split into train and test data sets based on whether the mood upon waking up is null.

In [161]:
known_mood = sleep_data_until_2019[pd.notna(sleep_data_until_2019['Wake up'])]
unknown_mood = sleep_data_until_2019[pd.isna(sleep_data_until_2019['Wake up'])]

In [162]:
known_mood

,Start,End,Sleep quality,Time in bed,Wake up,Heart rate,Activity (steps),Stressful day,Worked out,Drank tea,Drank coffee,Ate late
0,2014-12-29 22:57:49,2014-12-30 07:30:13,100.0,512,:),59.0,0,0,0,0,0,0
1,2014-12-30 21:17:50,2014-12-30 21:33:54,3.0,16,:|,72.0,0,1,0,0,0,0
2,2014-12-30 22:42:49,2014-12-31 07:13:31,98.0,510,:|,57.0,0,0,0,0,0,0
4,2015-01-01 22:12:10,2015-01-02 04:56:35,72.0,404,:),68.0,0,0,0,1,1,0
5,2015-01-03 00:34:57,2015-01-03 07:47:23,83.0,432,:),60.0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
243,2016-01-04 22:17:03,2016-01-05 05:45:29,74.0,448,:|,NaN,0,1,0,1,1,0
244,2016-01-05 22:18:45,2016-01-06 06:20:45,66.0,481,:|,NaN,0,0,1,0,1,0
245,2016-01-13 22:44:29,2016-01-14 06:20:25,72.0,455,:|,NaN,0,1,0,1,1,1
246,2016-01-14 22:10:58,2016-01-15 05:28:34,73.0,437,:|,NaN,0,0,1,1,1,0


Change the mood upon waking up into a quantitative variable.

In [163]:
one_hot = pd.get_dummies(known_mood['Wake up'])
known_mood = known_mood.drop('Wake up',axis = 1)
known_mood = known_mood.join(one_hot)

In [164]:
known_mood

,Start,End,Sleep quality,Time in bed,Heart rate,Activity (steps),Stressful day,Worked out,Drank tea,Drank coffee,Ate late,:(,:),:|
0,2014-12-29 22:57:49,2014-12-30 07:30:13,100.0,512,59.0,0,0,0,0,0,0,0,1,0
1,2014-12-30 21:17:50,2014-12-30 21:33:54,3.0,16,72.0,0,1,0,0,0,0,0,0,1
2,2014-12-30 22:42:49,2014-12-31 07:13:31,98.0,510,57.0,0,0,0,0,0,0,0,0,1
4,2015-01-01 22:12:10,2015-01-02 04:56:35,72.0,404,68.0,0,0,0,1,1,0,0,1,0
5,2015-01-03 00:34:57,2015-01-03 07:47:23,83.0,432,60.0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,2016-01-04 22:17:03,2016-01-05 05:45:29,74.0,448,NaN,0,1,0,1,1,0,0,0,1
244,2016-01-05 22:18:45,2016-01-06 06:20:45,66.0,481,NaN,0,0,1,0,1,0,0,0,1
245,2016-01-13 22:44:29,2016-01-14 06:20:25,72.0,455,NaN,0,1,0,1,1,1,0,0,1
246,2016-01-14 22:10:58,2016-01-15 05:28:34,73.0,437,NaN,0,0,1,1,1,0,0,0,1


In [165]:
known_mood['Wake up'] = known_mood[':|'] + known_mood[":)"] * 2
known_mood = known_mood.drop(':)',axis = 1)
known_mood = known_mood.drop(':(',axis = 1)
known_mood = known_mood.drop(':|',axis = 1)

In [166]:
known_mood

,Start,End,Sleep quality,Time in bed,Heart rate,Activity (steps),Stressful day,Worked out,Drank tea,Drank coffee,Ate late,Wake up
0,2014-12-29 22:57:49,2014-12-30 07:30:13,100.0,512,59.0,0,0,0,0,0,0,2
1,2014-12-30 21:17:50,2014-12-30 21:33:54,3.0,16,72.0,0,1,0,0,0,0,1
2,2014-12-30 22:42:49,2014-12-31 07:13:31,98.0,510,57.0,0,0,0,0,0,0,1
4,2015-01-01 22:12:10,2015-01-02 04:56:35,72.0,404,68.0,0,0,0,1,1,0,2
5,2015-01-03 00:34:57,2015-01-03 07:47:23,83.0,432,60.0,0,0,0,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
243,2016-01-04 22:17:03,2016-01-05 05:45:29,74.0,448,NaN,0,1,0,1,1,0,1
244,2016-01-05 22:18:45,2016-01-06 06:20:45,66.0,481,NaN,0,0,1,0,1,0,1
245,2016-01-13 22:44:29,2016-01-14 06:20:25,72.0,455,NaN,0,1,0,1,1,1,1
246,2016-01-14 22:10:58,2016-01-15 05:28:34,73.0,437,NaN,0,0,1,1,1,0,1


In [167]:
unknown_mood

,Start,End,Sleep quality,Time in bed,Wake up,Heart rate,Activity (steps),Stressful day,Worked out,Drank tea,Drank coffee,Ate late
3,2014-12-31 22:31:01,2015-01-01 06:03:01,65.0,452,NaN,NaN,0,0,0,0,0,0
6,2015-01-04 00:23:06,2015-01-04 07:37:09,78.0,434,NaN,NaN,0,0,0,1,0,0
248,2016-01-18 21:56:26,2016-01-19 05:34:59,78.0,458,NaN,NaN,0,0,0,1,1,1
249,2016-01-19 22:46:50,2016-01-20 06:35:37,75.0,468,NaN,NaN,0,0,1,1,1,0
250,2016-01-20 21:13:40,2016-01-21 06:29:37,77.0,555,NaN,NaN,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
882,2018-02-12 21:54:14,2018-02-13 07:02:15,91.0,548,NaN,NaN,56,0,0,0,0,0
883,2018-02-13 23:49:19,2018-02-14 07:00:53,81.0,431,NaN,NaN,64,0,0,0,0,0
884,2018-02-14 21:24:05,2018-02-15 06:20:52,71.0,536,NaN,NaN,3316,0,0,0,0,0
885,2018-02-15 21:36:32,2018-02-16 06:50:31,80.0,553,NaN,NaN,6555,0,0,0,0,0


In [168]:
known_mood = known_mood.fillna(known_mood.median())

/Users/arjunkavungal/Library/r-miniconda/envs/r-reticulate/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


In [169]:
known_mood

,Start,End,Sleep quality,Time in bed,Heart rate,Activity (steps),Stressful day,Worked out,Drank tea,Drank coffee,Ate late,Wake up
0,2014-12-29 22:57:49,2014-12-30 07:30:13,100.0,512,59.0,0,0,0,0,0,0,2
1,2014-12-30 21:17:50,2014-12-30 21:33:54,3.0,16,72.0,0,1,0,0,0,0,1
2,2014-12-30 22:42:49,2014-12-31 07:13:31,98.0,510,57.0,0,0,0,0,0,0,1
4,2015-01-01 22:12:10,2015-01-02 04:56:35,72.0,404,68.0,0,0,0,1,1,0,2
5,2015-01-03 00:34:57,2015-01-03 07:47:23,83.0,432,60.0,0,0,0,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
243,2016-01-04 22:17:03,2016-01-05 05:45:29,74.0,448,60.0,0,1,0,1,1,0,1
244,2016-01-05 22:18:45,2016-01-06 06:20:45,66.0,481,60.0,0,0,1,0,1,0,1
245,2016-01-13 22:44:29,2016-01-14 06:20:25,72.0,455,60.0,0,1,0,1,1,1,1
246,2016-01-14 22:10:58,2016-01-15 05:28:34,73.0,437,60.0,0,0,1,1,1,0,1


In [170]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer


# Prepare data for the model

In [177]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X = known_mood.iloc[:,2:-1].drop("Heart rate",axis=1)
y = known_mood['Wake up']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#scaler = MinMaxScaler().fit(X_train)
#quantile_transformer = QuantileTransformer(n_quantiles=10, random_state=0)
#X_train = quantile_transformer.fit_transform(X_train)
#X_train = scaler.transform(X_train)

## Put the model

In [178]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
accuracy_score(y_test, clf.predict(X_test))

0.86

In [179]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train,y_train)
accuracy_score(y_test, dtc.predict(X_test))

0.82

In [190]:
from sklearn.metrics import accuracy_score
dtc1 = DecisionTreeClassifier(random_state=11)
dtc1.fit(X_train,y_train)
accuracy_score(y_test, dtc1.predict(X_test))

0.84

In [183]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train, y_train)
accuracy_score(y_test, knn.predict(X_test))

0.86

In [182]:
from sklearn import svm
svc = svm.SVC()
svc.fit(X_train, y_train)
accuracy_score(y_test, svc.predict(X_test))

0.86

In [26]:
unknown_mood['Wake up'] = clf.predict(unknown_mood.iloc[:,2:].drop(["Heart rate","Wake up"],axis=1))

/Users/arjunkavungal/Library/r-miniconda/envs/r-reticulate/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
unknown_mood

,Start,End,Sleep quality,Time in bed,Wake up,Heart rate,Activity (steps),Stressful day,Worked out,Drank tea,Drank coffee,Ate late
3,2014-12-31 22:31:01,2015-01-01 06:03:01,65.0,452,2,NaN,0,0,0,0,0,0
6,2015-01-04 00:23:06,2015-01-04 07:37:09,78.0,434,2,NaN,0,0,0,1,0,0
248,2016-01-18 21:56:26,2016-01-19 05:34:59,78.0,458,2,NaN,0,0,0,1,1,1
249,2016-01-19 22:46:50,2016-01-20 06:35:37,75.0,468,2,NaN,0,0,1,1,1,0
250,2016-01-20 21:13:40,2016-01-21 06:29:37,77.0,555,2,NaN,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
882,2018-02-12 21:54:14,2018-02-13 07:02:15,91.0,548,2,NaN,56,0,0,0,0,0
883,2018-02-13 23:49:19,2018-02-14 07:00:53,81.0,431,2,NaN,64,0,0,0,0,0
884,2018-02-14 21:24:05,2018-02-15 06:20:52,71.0,536,2,NaN,3316,0,0,0,0,0
885,2018-02-15 21:36:32,2018-02-16 06:50:31,80.0,553,2,NaN,6555,0,0,0,0,0


In [28]:
unknown_mood['Heart rate'].unique()

array([nan])

In [29]:
sleep_data_from_2019 = pd.read_csv('sleepdata_2.csv',sep=";")

In [30]:
sleep_data_from_2019

,Start,End,Sleep Quality,Regularity,Mood,Heart rate (bpm),Steps,Alarm mode,Air Pressure (Pa),City,...,Time in bed (seconds),Time asleep (seconds),Time before sleep (seconds),Window start,Window stop,Did snore,Snore time,Weather temperature (°C),Weather type,Notes
0,2019-05-12 23:26:13,2019-05-13 06:11:03,60%,0%,NaN,0,8350,Normal,NaN,NaN,...,24289.2,22993.8,161.9,2019-05-13 06:00:00,2019-05-13 06:00:00,True,92.0,0.0,No weather,NaN
1,2019-05-13 22:10:31,2019-05-14 06:10:42,73%,0%,NaN,0,4746,Normal,NaN,NaN,...,28810.2,25160.9,192.1,2019-05-14 05:50:00,2019-05-14 05:50:00,True,0.0,0.0,No weather,NaN
2,2019-05-14 21:43:00,2019-05-15 06:10:41,86%,96%,NaN,0,4007,Normal,NaN,NaN,...,30461.5,28430.8,203.1,2019-05-15 05:50:00,2019-05-15 05:50:00,True,74.0,0.0,No weather,NaN
3,2019-05-15 23:11:51,2019-05-16 06:13:59,77%,92%,NaN,0,6578,Normal,NaN,NaN,...,25327.6,23132.5,168.9,2019-05-16 05:50:00,2019-05-16 05:50:00,True,0.0,0.0,No weather,NaN
4,2019-05-16 23:12:13,2019-05-17 06:20:32,78%,94%,NaN,0,4913,Normal,NaN,NaN,...,25698.4,22614.6,171.3,2019-05-17 05:50:00,2019-05-17 05:50:00,True,188.0,0.0,No weather,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,2022-03-24 21:33:46,2022-03-25 04:21:40,71%,77%,NaN,0,3903,Normal,95.2,Central Kootenay,...,24474.1,20803.0,489.5,2022-03-25 04:20:00,2022-03-25 04:20:00,True,695.7,4.0,Cloudy,NaN
917,2022-03-25 16:48:05,2022-03-25 17:30:22,9%,14%,NaN,0,495,Normal,83.8,North Okanagan,...,2536.2,0.0,0.0,2022-03-25 17:30:00,2022-03-25 17:30:00,True,0.0,5.0,Cloudy,NaN
918,2022-03-26 21:14:23,2022-03-27 06:11:01,49%,-1%,NaN,0,13388,Normal,83.5,North Okanagan,...,32198.1,24577.9,3649.1,2022-03-27 06:00:00,2022-03-27 06:00:00,True,506.4,-1.1,Cloudy,NaN
919,2022-03-28 22:53:23,2022-03-29 04:50:36,77%,22%,NaN,0,456,Normal,93.9,Central Kootenay,...,21433.6,15860.8,428.7,2022-03-29 04:20:00,2022-03-29 04:20:00,True,60.0,6.1,Sunny,NaN


In [31]:
sleep_data_from_2019.columns

Index(['Start', 'End', 'Sleep Quality', 'Regularity', 'Mood',
       'Heart rate (bpm)', 'Steps', 'Alarm mode', 'Air Pressure (Pa)', 'City',
       'Movements per hour', 'Time in bed (seconds)', 'Time asleep (seconds)',
       'Time before sleep (seconds)', 'Window start', 'Window stop',
       'Did snore', 'Snore time', 'Weather temperature (°C)', 'Weather type',
       'Notes'],
      dtype='object')

In [32]:
sleep_data_from_2019.Mood.unique()

array([nan])

In [198]:
sleep_data_from_2019['Notes'] = sleep_data_from_2019['Notes'].astype(str)

In [200]:
sleep_data_from_2019['Sleep Quality'] = sleep_data_from_2019['Sleep Quality'].astype(str).str.rstrip('%').astype('float')
sleep_data_from_2019['Time in bed'] = sleep_data_from_2019['Time in bed (seconds)'] / 60
sleep_data_from_2019['Sleep Notes'] = sleep_data_from_2019['Notes'].fillna("None")
sleep_data_from_2019['Stressful day'] = sleep_data_from_2019['Notes'].str.contains('Stressful day').astype(int)
sleep_data_from_2019['Worked out'] = sleep_data_from_2019['Notes'].str.contains('Worked out').astype(int)
sleep_data_from_2019['Drank tea'] = sleep_data_from_2019['Notes'].str.contains('Drank tea').astype(int)
sleep_data_from_2019['Drank coffee'] = sleep_data_from_2019['Notes'].str.contains('Drank coffee').astype(int)
sleep_data_from_2019['Ate late'] = sleep_data_from_2019['Notes'].str.contains('Ate late').astype(int)
sleep_data_from_2019 = sleep_data_from_2019.drop('Notes',axis = 1)

In [203]:
sleep_data_from_2019[['Sleep Quality','Time in bed','Steps','Stressful day','Worked out','Drank tea','Drank coffee','Ate late']]

,Sleep Quality,Time in bed,Steps,Stressful day,Worked out,Drank tea,Drank coffee,Ate late
0,60.0,404.820000,8350,0,0,0,0,0
1,73.0,480.170000,4746,0,0,0,0,0
2,86.0,507.691667,4007,0,0,0,0,0
3,77.0,422.126667,6578,0,0,0,0,0
4,78.0,428.306667,4913,0,0,0,0,0
...,...,...,...,...,...,...,...,...
916,71.0,407.901667,3903,0,0,0,0,0
917,9.0,42.270000,495,0,0,0,0,0
918,49.0,536.635000,13388,0,0,0,0,0
919,77.0,357.226667,456,0,0,0,0,0


In [210]:
sleep_data_from_2019['Mood'] = dtc.predict(sleep_data_from_2019[['Sleep Quality','Time in bed','Steps','Stressful day','Worked out','Drank tea','Drank coffee','Ate late']])

In [211]:
sleep_data_from_2019

,Start,End,Sleep Quality,Regularity,Mood,Heart rate (bpm),Steps,Alarm mode,Air Pressure (Pa),City,...,Weather temperature (°C),Weather type,Time in bed,Sleep Notes,Stressful day,Worked out,Drank tea,Drank coffee,Ate late,mood
0,2019-05-12 23:26:13,2019-05-13 06:11:03,60.0,0%,2,0,8350,Normal,NaN,NaN,...,0.0,No weather,404.820000,nan,0,0,0,0,0,2
1,2019-05-13 22:10:31,2019-05-14 06:10:42,73.0,0%,2,0,4746,Normal,NaN,NaN,...,0.0,No weather,480.170000,nan,0,0,0,0,0,2
2,2019-05-14 21:43:00,2019-05-15 06:10:41,86.0,96%,1,0,4007,Normal,NaN,NaN,...,0.0,No weather,507.691667,nan,0,0,0,0,0,1
3,2019-05-15 23:11:51,2019-05-16 06:13:59,77.0,92%,2,0,6578,Normal,NaN,NaN,...,0.0,No weather,422.126667,nan,0,0,0,0,0,2
4,2019-05-16 23:12:13,2019-05-17 06:20:32,78.0,94%,2,0,4913,Normal,NaN,NaN,...,0.0,No weather,428.306667,nan,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,2022-03-24 21:33:46,2022-03-25 04:21:40,71.0,77%,2,0,3903,Normal,95.2,Central Kootenay,...,4.0,Cloudy,407.901667,nan,0,0,0,0,0,2
917,2022-03-25 16:48:05,2022-03-25 17:30:22,9.0,14%,2,0,495,Normal,83.8,North Okanagan,...,5.0,Cloudy,42.270000,nan,0,0,0,0,0,2
918,2022-03-26 21:14:23,2022-03-27 06:11:01,49.0,-1%,1,0,13388,Normal,83.5,North Okanagan,...,-1.1,Cloudy,536.635000,nan,0,0,0,0,0,1
919,2022-03-28 22:53:23,2022-03-29 04:50:36,77.0,22%,0,0,456,Normal,93.9,Central Kootenay,...,6.1,Sunny,357.226667,nan,0,0,0,0,0,0
